* Anthem, indica los binding, no habia de necesidad de ver el threshold
* NetMHCpan4.1 indica que el threshold es 0.5 y ya decia los bindings
* Acme indica que > a 0.42 es el threshold
* MixMHCpred  no da un thhreshold, pero buscar el mejor para esta BD el mejor fue de 2.73083
* MHCflurry ,tampoco da un theshold

# prediciont Anthem

In [2]:
# read test data
import pandas as pd
import numpy as np

data = pd.read_csv("../dataset/hlab/hlab_test.csv")
data["id"] = list(range(1, data.shape[0]+1))

cols = data.columns.tolist()
cols.insert(0, cols.pop(cols.index('id')))
data = data.reindex(columns=cols)

print(data.head(2))
print(data.shape)

data.to_csv("../dataset/hlab/hlab_test2.csv", index=False)

   id          HLA   peptide  Label  Length  \
0   1  HLA-A*01:01  LFGRDLSY      1       8   
1   2  HLA-A*01:01  TDKKTHLY      1       8   

                                  mhc  
0  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
1  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
(172580, 6)


In [4]:
# group by peptide length (k-mer)
data_grouped = []  # cada elemento es un dataframe. 
# Ex: data_list[0] contiene los datos de peptidos de longitud 8
# Ex: data_list[1] contiene los datos de peptidos de longitud 9

for i in range(8,15):
    peptides = data[data['Length'] == i] 
    data_grouped.append(peptides)
    #peptides.to_csv(str(i) + "-mer.csv", index=False)

print(data_grouped[0].head(3)) # 8-mer peptides

   id          HLA   peptide  Label  Length  \
0   1  HLA-A*01:01  LFGRDLSY      1       8   
1   2  HLA-A*01:01  TDKKTHLY      1       8   
2   3  HLA-A*01:01  RSDTPLIY      1       8   

                                  mhc  
0  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
1  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
2  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  


In [ ]:
# por cada grupo k-mer, ahora separamos por el tipo de HLA
import os
import glob

def predict_anthem(hla, peptides):
    # save peptides in a tmp file
    tmp_peptide_file = open("temp_peptide_file.txt", "w")
    tmp_peptide_file.write("\n".join(peptides))
    tmp_peptide_file.close()

    length = str(len(peptides[0]))

    # cmd for anthem
    #python sware_b_main.py --length 9 --HLA HLA-A*01:01 --mode prediction --peptide_file test/predictpeptide.txt

    cmd = "cd Anthem; python sware_b_main.py --length "+length+" --HLA "+hla+" --mode prediction --peptide_file ../temp_peptide_file.txt"    
    os.system(cmd)
    results = get_last_folder_created()    
    files = glob.glob("Anthem/" + str(results) + "/*.txt")
    anthem_results = open(files[0], "r")
    lines = anthem_results.readlines() 
    lines = lines[5:len(lines)-3] # remove header and bottom
    bindings = []
    probs = []
    for line in lines: # procesamos cada peptido 
        tmp = line.split("\t")
        #print(tmp)
        peptide = tmp[3] # aqui esdta el peptido
        binding = 1 if tmp[5].strip() == "yes" else 0 # aqui esta si is binding
        prob = float(tmp[7].replace("\n", "").strip())
        
        bindings.append(binding)
        probs.append(prob)
        #print(peptide, binding)
    return bindings , probs  
    
    
    

# anthem genera los resultados en un folder, con esta funcion detectamos ese folder
def get_last_folder_created():
    folder = 'Anthem'
    possible_folder = [] # subfolder con nombre de fechas
    sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
    for folder in sub_folders:
        if folder.isnumeric():
            possible_folder.append(int(folder))
    return max(possible_folder)

  
final_results = pd.DataFrame(columns=['id','HLA','peptide','Label','Length','mhc','anthem_pred', 'anthem_prob'])
for df_by_kmer in data_grouped: # itereamos por cada k-mer
    print("generating...")
    hlas = df_by_kmer['HLA'].unique()
    for hla in hlas:
        df_by_hla = df_by_kmer[df_by_kmer['HLA'] == hla] # dividimos por tipo de hla
        peptides = df_by_hla["peptide"].tolist()     
        bindings, probs = predict_anthem(hla, peptides)   
        df_by_hla["anthem_pred"] = bindings
        df_by_hla["anthem_prob"] = probs
        final_results = pd.concat([final_results, df_by_hla]) 

        
    

#final_results.to_csv("results_tmp.csv", index=False)
print("finish :)")

In [7]:
final_results = final_results.sort_values('id')
final_results.to_csv("prediction_anthem.csv", index=False)

# Prediction with ESMt6

Vamos a aplicar softmax para normalizar los logits obtenidos en las predicciones de esm2t6 y tape

In [2]:
import pandas as pd
import numpy as np

def softmax(logits):
    return (np.exp(logits) / np.exp(logits).sum() )

#print(softmax([16.917, 0.772]))

#data = pd.read_csv("../predictions/30epochs_esm2_t6_freeze.csv", index_col=0)
data = pd.read_csv("/M2/ArgosMHC_models/predictions/distilbert_t33_c3.csv", index_col=0)
data['prob_0'] = data.apply(lambda row: ( softmax([row[0], row[1]])[0] ), axis=1)
data['prob_1'] = data.apply(lambda row: ( softmax([row[0], row[1]])[1] ), axis=1)
data.to_csv("predictions_esm2_t33_distil.csv")


In [3]:
#data = pd.read_csv("../predictions/30epochs_tape_acc_steps.csv", index_col=0)
data = pd.read_csv("/M2/ArgosMHC_models/predictions/tape_c3.csv", index_col=0)
data['prob_0'] = data.apply(lambda row: ( softmax([row[0], row[1]])[0] ), axis=1)
data['prob_1'] = data.apply(lambda row: ( softmax([row[0], row[1]])[1] ), axis=1)
data.to_csv("predictions_tape.csv")

In [9]:
print(data.head(10))

          0         1  prediction  label    prob_0    prob_1
0 -1.515012  2.123430           1      1  0.025620  0.974380
1 -2.883986  3.783111           1      1  0.001270  0.998730
2 -4.096594  5.074096           1      1  0.000104  0.999896
3 -3.912578  5.064070           1      1  0.000126  0.999874
4 -1.187903  2.257972           1      1  0.030892  0.969108
5  0.383407  0.175635           0      1  0.551757  0.448243
6 -1.899285  2.613402           1      1  0.010850  0.989150
7 -2.634839  3.608717           1      1  0.001939  0.998061
8 -1.740799  2.500628           1      1  0.014183  0.985817
9 -2.192737  3.056245           1      1  0.005225  0.994775


# Prediction with MixMHCpred

MixMHCpred es una herramienta top mejor en varios reviews: A comprehensive review and performance evaluation of bioinformatics tools for HLA class I peptide-binding prediction. Ahora en la version 2.2, es mejor: https://github.com/GfellerLab/MixMHCpred/tree/master

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("../dataset/hlab/hlab_test2.csv")
print(data.head(3))

In [ ]:
import os

final_results = pd.DataFrame(columns=['id','HLA','peptide','Label','Length','mhc','mixmhcpred_score', 'mixmhcpred_rank'])

def predict_mixmhcpred(hla,peptides):
    # create input file for Mixmhcpred
    inputfile = open("MixMHCpred/test/input.fa", "w")
    for pep in peptides:
        inputfile.write("> ESNT0000_" + pep + "\n" + pep + "\n")
    inputfile.close()

    # call MixMHCpred
    cmd = "cd MixMHCpred; ./MixMHCpred -i test/input.fa -o test/out.txt -a " + hla    
    os.system(cmd)

    # read output
    results = open("MixMHCpred/test/out.txt", "r")
    lines = results.readlines()
    results.close()
    data_results = lines[12:len(lines)]
    scores = []
    ranks = []
    for single_result in data_results:
        tmp = single_result.split("\t")        
        scores.append( float(tmp[1]) ) # get score
        ranks.append( float(tmp[3]) ) # get rank

    return scores,ranks

# Pra MixMHCpred, haremos las predicciones por HLA. Si le enviamos  varios hla, solo retorna con que hla hizo mejor el binding.
hlas = data['HLA'].unique()
for hla in hlas:
    print("processing...", hla)
    df_by_hla = data[data['HLA'] == hla] # dividimos por tipo de hla
    peptides = df_by_hla["peptide"].tolist()     
    scores,ranks = predict_mixmhcpred(hla, peptides)   
    df_by_hla["mixmhcpred_score"] = scores
    df_by_hla["mixmhcpred_rank"] = ranks
    final_results = pd.concat([final_results, df_by_hla]) 

    
    
final_results = final_results.sort_values('id')
final_results.to_csv("predictions_mixmhcpred.csv", index=False)
print("finish :)")

In [ ]:
# agregamos la prediccion de MicMHCpred
import pandas as pd

data = pd.read_csv("predictions_mixmhcpred.csv")
threshold = 2.73083 # es el mejor threshold para MixMHCpred, fue calculado en metrics
data['mixmhcpred_pred'] = data.apply(lambda row: ( 1 if row['mixmhcpred_rank'] < threshold else 0), axis=1)
print(data.head())

data.to_csv("predictions_mixmhcpred.csv", index=False)

# Prediction MHCFLurry

The last two columns give the antigen processing and presentation scores, respectively. These range from 0 to 1 with higher values indicating more favorable processing or presentation.

The processing predictor is experimental. It models allele-independent effects that influence whether a peptide will be detected in a mass spec experiment. The presentation score is a simple logistic regression model that combines the (log) binding affinity prediction with the processing score to give a composite prediction. The resulting prediction may be useful for prioritizing potential epitopes, but no thresholds have been established for what constitutes a “high enough” presentation score.

https://openvax.github.io/mhcflurry/commandline_tutorial.html#generating-predictions

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("../dataset/hlab/hlab_test2.csv")
print(data.head(3))

   id          HLA   peptide  Label  Length  \
0   1  HLA-A*01:01  LFGRDLSY      1       8   
1   2  HLA-A*01:01  TDKKTHLY      1       8   
2   3  HLA-A*01:01  RSDTPLIY      1       8   

                                  mhc  
0  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
1  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  
2  YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY  


In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
import logging
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

final_results = pd.DataFrame(columns=['id','HLA','peptide','Label','Length','mhc'])

def predict_mhcflurry(hla,peptides):
    print(len(peptides))
    pep_str = " ".join(peptides)    

    # call MixMHCpred
    cmd = "mhcflurry-predict --alleles " + hla + " --peptides " + pep_str + " --out tmp_predictions_mhcflurry.csv "    
    os.system(cmd)

    # read output
    predictions = pd.read_csv("tmp_predictions_mhcflurry.csv")

    

    return  predictions["mhcflurry_affinity"], predictions["mhcflurry_affinity_percentile"], predictions["mhcflurry_processing_score"], predictions["mhcflurry_presentation_score"], predictions["mhcflurry_presentation_percentile"]

# Pra MHCFlurry, haremos las predicciones por HLA. 
hlas = data['HLA'].unique()
#print(hlas)
for hla in hlas:
    print("processing...", hla)

    #if hla != "HLA-A*29:02": continue


    df_by_hla = data[data['HLA'] == hla] # dividimos por tipo de hla
    peptides = df_by_hla["peptide"].tolist()     
    aff, aff_per, score, present, present_per  = predict_mhcflurry(hla, peptides)  
    
    df_by_hla["affinity"] = aff.tolist() 
    df_by_hla["affinity_percentile"] = aff_per.tolist() 
    df_by_hla["processing_score"] = score.tolist() 
    df_by_hla["presentation_score"] = present.tolist() 
    df_by_hla["presentation_percentile"] = present_per.tolist() 
    print(df_by_hla.head(10)) # x motivos raros, al agregar esto se arreglo. al princpio, no copiaba la infomracion de los dataframes.
    final_results = pd.concat([final_results, df_by_hla])   

    
    
final_results = final_results.sort_values('id')
final_results.to_csv("predictions_mhcflurry.csv", index=False)
print("finish :)")

In [14]:
# agregamos la prediccion de MicMHCpred
import pandas as pd
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    auc, roc_curve,
    matthews_corrcoef
)
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import numpy as np

data = pd.read_csv("predictions_mhcflurry.csv")


fpr, tpr, thresholds = roc_curve(data['Label'], data['presentation_score'], pos_label = 1)
print(len(thresholds), max(thresholds), min(thresholds), sum(thresholds)/len(thresholds))
print(thresholds[2000:2030])

# IMPORTANTE, CON ESTO SABEMOS CUAL ES EL BEST THRESHOLD
print("best threshold", thresholds[np.argmax(tpr - fpr)])
auc_val = auc(fpr, tpr)    
print("AUC:", auc_val)

threshold = 0.094388919139774 # es el mejor threshold para MHCflurry
data['mhcflurry_prob'] = data['presentation_score']
data['mhcflurry_pred'] = data.apply(lambda row: ( 1 if row['presentation_score'] > threshold else 0), axis=1)
#print(data.head())

data.to_csv("predictions_mhcflurry.csv", index=False)

23886 inf 0.0029054224539779 inf
[0.72295951 0.72295824 0.72277737 0.72275643 0.72268608 0.72268076
 0.72260864 0.7225967  0.72218884 0.7221867  0.72196637 0.72196445
 0.72154094 0.72152752 0.72090448 0.72090366 0.72057499 0.7205681
 0.72055706 0.72047599 0.72042397 0.72040619 0.72039148 0.72036535
 0.7203087  0.72030771 0.72002765 0.71999193 0.71998104 0.7199552 ]
best threshold 0.094388919139774
AUC: 0.9641677983696788
